In [1]:
import collections
import logging
import math
import os
import warnings
from timeit import default_timer as timer

import wandb
from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    LineByLineTextDataset,
    TextDataset,
    Trainer,
    TrainingArguments,
    set_seed,
)

warnings.filterwarnings("ignore")
wandb.login()

start = "<|startoftext|>"
sep = "<|sep|>"


def dict2obj(d):
    """Convert a dictionary to a class"""
    if isinstance(d, list):
        d = [dict2obj(x) for x in d]
    if not isinstance(d, dict):
        return d

    class Class:
        pass

    obj = Class()
    for k in d:
        obj.__dict__[k] = dict2obj(d[k])
    return obj


def get_dataset(args, tokenizer, evaluate=False):
    file_path = args.eval_data_file if evaluate else args.train_data_file
    if args.line_by_line:
        return LineByLineTextDataset(
            tokenizer=tokenizer, file_path=file_path, block_size=args.block_size
        )
    else:
        return TextDataset(
            tokenizer=tokenizer,
            file_path=file_path,
            block_size=args.block_size,
            overwrite_cache=args.overwrite_cache,
        )




Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chaowan (use `wandb login --relogin` to force relogin)


In [ ]:
# create a smaller dataset for training and validation
count=0
path_full = "hep_full_dataset/train_ml_full.txt"
path_part = "hep_full_dataset/train_ml_50000.txt"
with open(path_full,"r") as full:   
    with open(path_part,"w+") as part:
        lines = full.readlines() 
        for line in lines:
            if count<50000:
                count+=1
                part.write(f"{line}\n")
    part.close()
full.close()

print(f"{count} training points created!")

# create a smaller dataset for training and validation
count=0
path_full = "hep_full_dataset/valid_hep_full.txt"
path_part = "hep_full_dataset/valid_hep_2000.txt"
with open(path_full,"r") as full:   
    with open(path_part,"w+") as part:
        lines = full.readlines() 
        for line in lines:
            if count<2000:
                count+=1
                part.write(f"{line}\n")
    part.close()
full.close()

print(f"{count} validation points created!")

In [2]:
# Model arguments
model_args = collections.defaultdict(
    config_name="gpt2",
    model_name_or_path="gpt2",
    model_type="gpt2",
    tokenizer_name="gpt2",
    cache_dir=None,
)
# Data arguments
data_args = collections.defaultdict(
    train_data_file="ml_full_dataset/train_ml_full.txt",
    eval_data_file="ml_full_dataset/valid_ml_full.txt",
    line_by_line=True,
    mlm=False,
    mlm_probability=0.15,
    block_size=512,
    overwrite_cache=False,
)

# Convert dict to objects
model_args = dict2obj(model_args)
data_args = dict2obj(data_args)




In [3]:
# Load tokenizer and model
config = AutoConfig.from_pretrained(
    model_args.model_name_or_path, cache_dir=model_args.cache_dir
)


tokenizer = AutoTokenizer.from_pretrained(
    model_args.model_name_or_path, cache_dir=model_args.cache_dir
)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    
tokenizer.pad_token = "[PAD]"
    
    
model = AutoModelWithLMHead.from_pretrained(
    model_args.model_name_or_path,
    from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
    cache_dir=model_args.cache_dir,
)

print("model loaded")

# Add special tokens
tokenizer.add_special_tokens({"sep_token": sep})
tokenizer.add_special_tokens({"bos_token": start})
model.resize_token_embeddings(len(tokenizer))

if tokenizer.pad_token is None:
    print("no pad token")
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

print("special tokens added")

Using pad_token, but it is not set yet.


model loaded
special tokens added


In [4]:
# Logging
logger = logging.getLogger(__name__)
# Model classes
MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)
# Training arguments
training_args = TrainingArguments(
    output_dir="model_ml_full_1/",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=False,
#    evaluate_during_training=True,
    per_gpu_train_batch_size=4,
    per_gpu_eval_batch_size=4,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    weight_decay=0.0,
    adam_epsilon=1e-08,
    max_grad_norm=1.0,
    num_train_epochs=4.0,
    max_steps=-1,
    warmup_steps=0,
    logging_dir=None,
    logging_first_step=False,
    logging_steps=1000,
    save_steps=10000,
    save_total_limit=100000,
    no_cuda=False,
    seed=123,
    fp16=False,
    fp16_opt_level="O1",
    local_rank=-1,
)

In [5]:
# Load dataset
train_dataset = (
    get_dataset(data_args, tokenizer=tokenizer) if training_args.do_train else None
)

print("train set loaded")


train set loaded


In [6]:
eval_dataset = (
    get_dataset(data_args, tokenizer=tokenizer, evaluate=True)
    if training_args.do_eval
    else None
)

print("eval set loaded")

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=data_args.mlm, mlm_probability=data_args.mlm_probability,
)


eval set loaded


In [7]:

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
#    prediction_loss_only=True,
)

print("initialized trainer")

# Logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
)
logger.warning(
    "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
    training_args.local_rank,
    training_args.device,
    training_args.n_gpu,
    bool(training_args.local_rank != -1),
    training_args.fp16,
)
logger.info("Training/evaluation parameters %s", training_args)

# Seed
set_seed(training_args.seed)


03/28/2022 13:27:14 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 2, distributed training: False, 16-bits training: False
03/28/2022 13:27:14 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=2,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
l

initialized trainer


In [8]:
# Define model path
model_path = (
    model_args.model_name_or_path
    if model_args.model_name_or_path is not None
    and os.path.isdir(model_args.model_name_or_path)
    else None
)

print("model path defined")

model path defined


In [ ]:
print("start Training")
# Train the model
start = timer()
train_results = trainer.train(model_path=model_path)
end = timer()
print("Training completed")

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 182764
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 91384
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.


start Training
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Step,Training Loss


In [ ]:
trainer.save_model()
print("model saved")


if trainer.is_world_process_zero():
    tokenizer.save_pretrained(training_args.output_dir)

# Calculate training time
logger.info(f"Training took {(end - start) / 3600} hours.")

# Evaluation on validation set
logger.info("*** Valid Evaluate ***")
valid_eval_output = trainer.evaluate()
valid_perplexity = math.exp(valid_eval_output["eval_loss"])
valid_result = {"valid_perplexity": valid_perplexity}
output_eval_file = os.path.join(training_args.output_dir, "valid_eval_results_lm.txt")

with open(output_eval_file, "w") as writer:
    logger.info("***** Valid Eval results *****")
    for key in sorted(valid_result.keys()):
        logger.info("  %s = %s", key, str(valid_result[key]))
        writer.write("%s = %s\n" % (key, str(valid_result[key])))

In [ ]:

# Evaluation on test set
training_args.do_eval = True
data_args.eval_data_file = "/home/cw862/arxiv/ml_full_dataset/test_ml_full.txt"
test_dataset = (
    get_dataset(data_args, tokenizer=tokenizer, evaluate=True)
    if training_args.do_eval
    else None
)
trainer.eval_dataset = test_dataset

logger.info("*** Test Evaluate ***")
test_eval_output = trainer.evaluate()
test_perplexity = math.exp(test_eval_output["eval_loss"])
test_result = {"test_perplexity": test_perplexity}
output_eval_file = os.path.join(training_args.output_dir, "test_eval_results_lm.txt")

with open(output_eval_file, "w") as writer:
    logger.info("***** Test Eval results *****")
    for key in sorted(test_result.keys()):
        logger.info("  %s = %s", key, str(test_result[key]))
        writer.write("%s = %s\n" % (key, str(test_result[key])))


# Evaluation on training set
data_args.eval_data_file = "/home/cw862/arxiv/AI_full_dataset/train_ai_all.txt"
test_dataset = (
    get_dataset(data_args, tokenizer=tokenizer, evaluate=True)
    if training_args.do_eval
    else None
)
trainer.eval_dataset = test_dataset

logger.info("*** Train Evaluate ***")
train_eval_output = trainer.evaluate()
train_perplexity = math.exp(train_eval_output["eval_loss"])
train_result = {"train_perplexity": train_perplexity}
output_eval_file = os.path.join(training_args.output_dir, "train_eval_results_lm.txt")

with open(output_eval_file, "w") as writer:
    logger.info("***** Train Eval results *****")
    for key in sorted(train_result.keys()):
        logger.info("  %s = %s", key, str(train_result[key]))
        writer.write("%s = %s\n" % (key, str(train_result[key])))


print(f"Train loss: {train_eval_output['eval_loss']}")
print(f"Valid loss: {valid_eval_output['eval_loss']}")
print(f"Test loss: {test_eval_output['eval_loss']}")
print(f"Train PPL: {train_perplexity}")
print(f"Valid PPL: {valid_perplexity}")
print(f"Test PPL: {test_perplexity}")